In [2]:
import saspy
import time

In [3]:
sas = saspy.SASsession()
# ll = sas.submit(f'libname db "/data/daveting/dataset/Compustat/d_na";')
# ll = sas.submit(f'libname db "/data/daveting/dataset/Compustat/d_na/security";')
# ll = sas.submit(f'libname db "/data/daveting/dataset/Compustat/d_global";')
# ll = sas.submit(f'libname db "/data/daveting/dataset/Compustat/d_global/security";')
# ll = sas.submit(f'libname db "/data/daveting/dataset/Refinitiv/ds_eq/reference_tables";')
# ll = sas.submit(f'libname db "/data/daveting/dataset/Refinitiv/esg";')
# ll = sas.submit(f'libname db "/data/daveting/dataset/Trucost/common";')
ll = sas.submit(f'libname db "/data/daveting/dataset/Capital_IQ/helper";')

Using SAS Config named: default
SAS Connection established. Subprocess id is 177334



In [22]:
ck = pd.read_parquet('/data/daveting/dataset/Std_Security_Code/isin/ciq_company_id.pq').drop(columns=['source', 'update'])
ck.shape
ck.head(3)

(9600285, 2)

,isin,ciq_company_id
4800142,XS1730704464,26081567
6400203,CH0379055210,34418814
6400187,CH0379055012,34418814


In [164]:
sk = pd.read_parquet('/data/daveting/dataset/Std_Security_Code/isin/company_name.pq')
# sk.drop(columns=['iid'], inplace=True)
sk.shape
sk.head(3)

(10222901, 4)

,isin,company_name,source,update
10222900,CNE100000TP3,LUXSHARE PRECISION INDUSTRY CO LTD (LUXSHARE),RepRisk,2022-06-19
10040426,US091096EA16,WATER WORKS BOARD OF THE CITY OF BIRMINGHAM; T...,RepRisk,2022-06-19
10040440,US091096DL89,WATER WORKS BOARD OF THE CITY OF BIRMINGHAM; T...,RepRisk,2022-06-19


In [175]:
### 读取CSV/Parquet源文件
# df = pd.read_csv("/data/daveting/dataset/Sustainalytics/hist_weighted_score.csv", encoding='latin1', low_memory=False,
#                  usecols=["ISIN", 'Ticker'],
#                  # nrows=5,
#                 )
df = pd.read_parquet("/data/daveting/dataset/RepRisk/2022upd/pm_company_identifiers.pq",
                 # columns=["name", 'primary_ISIN'],
                 # nrows=5,
                ) #.dropna()
# Sustainalytics: ISIN: Company CapitalIQ_ID Ticker
df.shape
df.sample(3)

(181582, 9)

,RepRisk_ID,name,headquarter_country,headquarter_country_code,sectors,url,all_ISINs,primary_ISIN,No_reported_risk_exposure
110693,295056,Aravali Power Co Pvt Ltd (APCPL),India,IN,Utilities,http://www.apcpl.co.in/,None,None,False
173261,86231,Belgorodenergo OJSC,Russian Federation,RU,Utilities,None,None,RU000A0F7CY4,False
158967,644640,Sysco Eastern Maryland LLC,United States of America,US,Food and Beverage | Retail,https://sysco.com/Contact/Contact/Our-Location...,None,None,False


In [166]:
### RepRisk堆叠特殊处理
# df['all_ISINs'] = df['all_ISINs'].str.split(' ')
# df = df.explode('all_ISINs')
# df.sample()

In [120]:
### 读取SAS源文件
df = sas.sd2df(
    libref='db', 
    table='wrds_ticker', 
    dsopts={
        'keep': ['companyid', 'ticker'],
        # comp names: isin(x)
        # comp security: isin: tic gvkey-iid cusip ibtic sedol
        # comp g_names: isin: gvkey conm sedol
        # comp g_security: isin: tic(x) gvkey-iid cusip(x) ibtic sedol
        # tfn wrds_ds_names: ISIN ISIN2: InfoCode DsCode DsSecCode(x) DsQtName(x) DsLocalCode(x)
        #                            DsMnem(Ticker?) PermID(x) DsSctyCode(x) DsCmpyCode(x) DsSecName(x) PrimQtSedol PrimQtInfoCode
        #                            WSSctyPPI IBESTicker WSSctyPPI2 IBESTicker2 DsCompCode(x) DsCmpyName
        # tfn wrds_company_info: Isin: OrgID Asset4Id(x) OrgName tick Cusip Sedol
        # trucost wrds_companies: gvkey: ticker companyname institutionid
        # ciq wrds_isin: companyid: isin companyname
        # ciq wrds_gvkey: companyid: gvkey companyname
        # ciq wrds_cusip: companyid: cusip companyname
        # ciq wrds_cik: companyid: cik companyname
        # ciq wrds_ciqsymbol: companyid: 所有symbol的link table
        # ciq ciqgvkeyiid: gvkey iid relatedcompanyid
        # ciq wrds_ticker: companyid: ticker companyname
        # 'obs': 10,
    }
)
df.shape
df.head()
df.isna().sum()

(4570046, 2)

,companyid,ticker
0,18507.0,1026283
1,18507.0,MMINV
2,18507.0,TW8
3,18507.0,MMINVN
4,18507.0,1026852


companyid     0
ticker       25
dtype: int64

In [167]:
df.dropna(inplace=True)
df.shape
df.rename(columns={'ibtic': 'ibes_ticker', 
                   'IBESTicker': 'ibes_ticker', 
                   'IBESTicker2': 'ibes_ticker', 
                   
                   'tic': 'ticker', 
                   'tick': 'ticker',
                   'Ticker': 'ticker',
                   
                   'Cusip': 'cusip',
                   
                   'conm': 'company_name', 
                   'DsCmpyName': 'company_name',
                   'OrgName': 'company_name',
                   'companyname': 'company_name',
                   'Company': 'company_name',
                   'name': 'company_name',
                   
                   'ISIN': 'isin', 
                   'ISIN2': 'isin',
                   'Isin': 'isin',
                   'primary_ISIN': 'isin',
                   'all_ISINs': 'isin',
                   
                   'InfoCode': 'ds_infocode',
                   'PrimQtInfoCode': 'ds_infocode',
                   
                   'DsCode': 'ds_code', 
                   
                   'PrimQtSedol': 'sedol',
                   'Sedol': 'sedol',

                   'WSSctyPPI': 'ws_sctyppi',
                   'WSSctyPPI2': 'ws_sctyppi',
                   
                   'OrgID': 'a4_orgid',
                   
                   'institutionid': 'sp_institutionid', 
                   
                   'CapitalIQ_ID': 'ciq_company_id',
                   'companyid': 'ciq_company_id', 
                   'relatedcompanyid': 'ciq_company_id', 
                   "RepRisk_ID": 'reprisk_id', }, inplace=True)
df.drop_duplicates(inplace=True)
df.shape
df.head()

(20829, 2)

(20829, 2)

,company_name,isin
0,Acer Inc,TW0002353000
1,Rio Tinto PLC,GB0007188757
2,Terrane Metals Corp,CA88103A1084
3,BLUECOM Co Ltd,KR7033560004
13,Hytera Communications Corp Ltd,CNE1000013B1


In [168]:
if 'ibes_ticker' in df.columns: df['ibes_ticker'] = df['ibes_ticker'].map(str).str.replace('@:', '')
# if 'ciq_company_id' in df.columns: df['ciq_company_id'] = df['ciq_company_id'].str.replace('IQ', '')

for var in ['sedol', 'cusip', 'iid', 'ds_code', 'ws_sctyppi', 'company_name', 'ticker', 'ibes_ticker']:
    if var in df.columns: df[var] = df[var].str.upper()

for var in ['gvkey', 'ds_infocode', 'a4_orgid', 'sp_institutionid', 'ciq_company_id', 'cik', 'reprisk_id']:
    if var in df.columns: df[var] = df[var].map(int)

df.head()
df.dtypes

,company_name,isin
0,ACER INC,TW0002353000
1,RIO TINTO PLC,GB0007188757
2,TERRANE METALS CORP,CA88103A1084
3,BLUECOM CO LTD,KR7033560004
13,HYTERA COMMUNICATIONS CORP LTD,CNE1000013B1


company_name    object
isin            object
dtype: object

In [148]:
### 公司-证券过桥
df = ck.merge(df, on=['ciq_company_id'], how='inner')[['isin', 'ticker']]
df.shape
df.head()

In [134]:
### 新建数据
df['source'] = 'RepRisk'
# 'Capital IQ'  'Sustainalytics'  'S&P ESG'  'Refinitiv ESG'  'Refinitiv Datastream'  'Compustat Global'
df['update'] = pd.to_datetime('2022-06-19')
dfm = df.copy()

In [169]:
dfm = sk.merge(df, on=['isin', 'company_name'], how='outer', indicator=True)
dfm['_merge'].value_counts()
dfm.shape
dfm.head()

left_only     10203772
both             19129
right_only        1700
Name: _merge, dtype: int64

(10224601, 5)

,isin,company_name,source,update,_merge
0,CNE100000TP3,LUXSHARE PRECISION INDUSTRY CO LTD (LUXSHARE),RepRisk,2022-06-19,both
1,US091096EA16,WATER WORKS BOARD OF THE CITY OF BIRMINGHAM; T...,RepRisk,2022-06-19,left_only
2,US091096DL89,WATER WORKS BOARD OF THE CITY OF BIRMINGHAM; T...,RepRisk,2022-06-19,left_only
3,US091096DM62,WATER WORKS BOARD OF THE CITY OF BIRMINGHAM; T...,RepRisk,2022-06-19,left_only
4,US091096DN46,WATER WORKS BOARD OF THE CITY OF BIRMINGHAM; T...,RepRisk,2022-06-19,left_only


In [170]:
dfm.loc[dfm['_merge'] == 'right_only', 'source'] = 'RepRisk'
# 'Capital IQ'  'Compustat NA'  'Compustat Global'  'Refinitiv Datastream'  'Refinitiv ESG'  'S&P ESG'  'Sustainalytics'
dfm.loc[dfm['_merge'] == 'right_only', 'update'] = pd.to_datetime('2022-06-19')
# dfm.loc[dfm['_merge'] == 'right_only'].sample(2)

In [171]:
dfm = dfm.loc[:, ['isin', 'company_name', 'source', 'update']]
dfm.shape
dfm.sample(3)

(10224601, 4)

,isin,company_name,source,update
7967839,GB00BHLLCG05,LLOYDS BANK PLC,Capital IQ,2022-06-18
8210909,US22546KCU25,CREDIT SUISSE INTERNATIONAL,Capital IQ,2022-06-18
6602358,DE000CU2UBZ0,COMMERZBANK AG,Capital IQ,2022-06-18


In [172]:
dfm.duplicated(subset=['isin', 'company_name']).sum()
dfm.sort_values(by=['update'], ascending=False, inplace=True)
dfm.drop_duplicates(subset=['isin', 'company_name'], keep='first', inplace=True)
dfm.shape

0

(10224601, 4)

In [173]:
### 隔离，输出前请检查

In [174]:
time.sleep(10)  # 延迟十秒才输出
dfm.to_parquet('/data/daveting/dataset/Std_Security_Code/isin/company_name.pq')